In [1]:
import pandas as pd
import numpy as np
import gcsfs
import sys
import xarray as xr
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)

<ipython-input-1-c85a82a673b8>:9: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


In [2]:
df = pd.read_csv('https://cmip6.storage.googleapis.com/cmip6-zarr-consolidated-stores.csv')

In [3]:
# define a simple search on keywords
def search_df(df, verbose= False, **search):
    "search by keywords - if list, then match exactly, otherwise match as substring"
    keys = ['activity_id','institution_id','source_id','experiment_id','member_id', 'table_id', 'variable_id', 'grid_label']
    d = df
    for skey in search.keys():
        
        if isinstance(search[skey], str):  # match a string as a substring
            d = d[d[skey].str.contains(search[skey])]
        else:
            dk = []
            for key in search[skey]:       # match a list of strings exactly
                dk += [d[d[skey]==key]]
            d = pd.concat(dk)
            keys.remove(skey)
    if verbose:
        for key in keys:
            print(key,' = ',list(d[key].unique()))      
    return d

In [4]:
#dfp = search_df(df, experiment_id=['historical','ssp585'], table_id=['Omon','Amon'], variable_id=["uas","vas","tos"], grid_label=['gr'])
dfp = search_df(df,
                experiment_id=["historical","ssp585","ssp245"],
                table_id=["Amon","Amon","Amon","Amon","Omon","SImon", "SImon","SImon","SImon"],
                variable_id=["uas","vas","tas","clt","chl","sithick", "siconc", "sisnthick", "sisnconc"],
                grid_label=['gn'])
#dfp = search_df(df, experiment_id=['historical','ssp585'], member_id=['r1i1p1f1'],table_id=['Amon','SImon','Omon','Amon'], variable_id=["uas","vas","chl","clt"], grid_label=['gn'])


In [5]:
dm = dfp[["experiment_id","source_id","variable_id","member_id","grid_label"]].groupby(["member_id","source_id","experiment_id","grid_label"]).nunique()[["variable_id"]]

In [6]:
#dm2=dm[dm["experiment_id"] in dm["historical","ssp585"]]
#print(dm2)
table = pd.DataFrame.pivot_table(dm, values="variable_id", index=["source_id","member_id"],
                                 columns=["grid_label","experiment_id"], aggfunc=np.sum, fill_value=0)

table[(table.values>=9)]

grid_label                        gn              
experiment_id             historical ssp245 ssp585
source_id       member_id                         
ACCESS-ESM1-5   r10i1p1f1  5          9      9    
                r10i1p1f1  5          9      9    
                r11i1p1f1  9          4      0    
                r12i1p1f1  9          5      0    
                r13i1p1f1  9          7      0    
                r14i1p1f1  9          2      0    
                r15i1p1f1  9          3      0    
                r16i1p1f1  9          1      0    
                r17i1p1f1  9          3      0    
                r18i1p1f1  9          2      0    
                r19i1p1f1  9          5      0    
                r1i1p1f1   9          9      9    
                r1i1p1f1   9          9      9    
                r1i1p1f1   9          9      9    
                r20i1p1f1  9          1      0    
                r2i1p1f1   9          9      9    
                r2i1p1f1   9          9      9    
                r2i1p1f1   9          9      9    
                r3i1p1f1   9          9      9    
                r3i1p1f1   9          9      9    
                r3i1p1f1   9          9      9    
                r4i1p1f1   9          9      9    
                r4i1p1f1   9          9      9    
                r4i1p1f1   9          9      9    
                r5i1p1f1   9          9      9    
                r5i1p1f1   9          9      9    
                r5i1p1f1   9          9      9    
                r6i1p1f1   9          8      9    
                r6i1p1f1   9          8      9    
                r7i1p1f1   8          9      9    
                r7i1p1f1   8          9      9    
                r8i1p1f1   5          9      9    
                r8i1p1f1   5          9      9    
                r9i1p1f1   5          8      9    
CMCC-ESM2       r1i1p1f1   9          6      7    
CanESM5         r10i1p2f1  9          9      9    
                r10i1p2f1  9          9      9    
                r10i1p2f1  9          9      9    
                r1i1p2f1   9          9      9    
                r1i1p2f1   9          9      9    
                r1i1p2f1   9          9      9    
                r2i1p2f1   9          9      9    
                r2i1p2f1   9          9      9    
                r2i1p2f1   9          9      9    
                r3i1p2f1   9          9      9    
                r3i1p2f1   9          9      9    
                r3i1p2f1   9          9      9    
                r4i1p2f1   9          9      9    
                r4i1p2f1   9          9      9    
                r4i1p2f1   9          9      9    
                r5i1p2f1   9          9      9    
                r5i1p2f1   9          9      9    
                r5i1p2f1   9          9      9    
                r6i1p2f1   9          9      9    
                r6i1p2f1   9          9      9    
                r6i1p2f1   9          9      9    
                r7i1p2f1   9          9      9    
                r7i1p2f1   9          9      9    
                r7i1p2f1   9          9      9    
                r8i1p2f1   9          9      9    
                r8i1p2f1   9          9      9    
                r8i1p2f1   9          9      9    
                r9i1p2f1   9          9      9    
                r9i1p2f1   9          9      9    
                r9i1p2f1   9          9      9    
MPI-ESM-1-2-HAM r1i1p1f1   9          0      0    
                r2i1p1f1   9          0      0    
                r3i1p1f1   9          0      0    
MPI-ESM1-2-HR   r10i1p1f1  9          0      0    
                r1i1p1f1   9          9      9    
                r1i1p1f1   9          9      9    
                r1i1p1f1   9          9      9    
                r2i1p1f1   9          9      9    
                r2i1p1f1   9          9      9    
                r2i1p1f1   9          9      9    
                r3i1p1

In [7]:
#table[(table.uas>0)&(table.vas>0)&(table.chl>0)&(table.clt>0)&(table.sisnconc>0)&(table.sisnthick>0)&(table.siconc>0)&(table.sithick>0)]
table[(table.sithick>0)&(table.siconc>0)&(table.sisnconc>0)&(table.sisnthick>0)]
print(dm)


AttributeError: 'DataFrame' object has no attribute 'sithick'